## 표본추출
### 
### 단순임의추출 (랜덤추출)
### ```sample(x, size, replace, prob)``` : x에서 size만큼 랜덤추출
- ```replace``` : 복원추출 여부 (TRUE : 복원 / FALSE : 비복원)

### 
### 층화추출 : 모집단이 이질적인 몇 개의 계층으로 이루어져 있을 때 모든 계층으로부터 원소를 임의로 추출하여 각 계층을 고루 대표할 수 있도록 랜덤하게 표본을 추출

### K-계통추출 : K개씩 n개의 구간으로 분할하고, 첫 구간에서 하나를 임의로 선택한 후에 K의 간격으로 표본을 선택. 총 n개의 표본

### ```sampling::strata(data, size, stratanames, method)``` : sampling 객체 생성
- ```stratanames``` : 층화 구분 변수
- ```size``` : 각 층에서 추출할 데이터 개수
- ```method``` : 데이터 추출 방법 (```"srswor"``` : 비복원 랜덤 / ```"srswr"``` : 복원 랜덤 / ```"poisson"``` : 포아송 추출 / ```"systematic"``` : 계통 추출)

### ```sampling::getdata(data, m)``` : sampling 객체를 대입하여 표본 추출
- ```m``` : ```sampling::strata()```로 생성한 sampling 객체

In [6]:
library(tidyverse)
library(sampling)

In [18]:
iris$Species %>% summary

setosa versicolor  virginica 
        50         50         50

In [22]:
sample <- sampling::strata(data = iris, size = c(20, 15, 15), stratanames = "Species", method = "srswor")
iris_sample <- getdata(iris, sample)

In [26]:
iris_sample$Species %>% summary
iris_sample %>% head(2)

setosa versicolor  virginica 
        20         15         15

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,ID_unit,Prob,Stratum
,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<int>,<dbl>,<int>
6,5.4,3.9,1.7,0.4,setosa,6,0.4,1
8,5.0,3.4,1.5,0.2,setosa,8,0.4,1


### 
### 군집추출법 : 군집을 구분하고 특정 군집을 먼저 선택한 후 해당 군집에서만 표본추출

### 

### ```rsample::initial_split(data, strata, prop)``` : 데이터의 strata의 비율을 유지한 채, prop만큼 훈련 / 평가 데이터 분할
- ```strata``` : 층화 구분 변수
- ```prop``` : 훈련 / 평가 데이터 비율

### ```rsample::training(split)``` / ```rsample::testing(split)```

In [27]:
library(rsample)

In [28]:
split <- initial_split(iris, strata = "Species", prop = 0.7)
train <- training(split) ; test <- testing(split)
train %>% dim ; train$Species %>% summary ; test %>% dim ; test$Species %>% summary

[1] 105   5

setosa versicolor  virginica 
        35         35         35

[1] 45  5

setosa versicolor  virginica 
        15         15         15

### 
## 일표본 검정

In [110]:
data <- read.csv("C:/Soongsil/R/빅데이터 R프로그래밍/Rwork-2nd/Part-III/one_sample.csv")
data %>% dim

[1] 150   4

### 단일집단 비율검정
### 이항분포 비율검정 : 관측된 확률이 ${x \over n}$인 데이터의 실제확률이 p인지 검정

- **귀무가설 : 데이터의 실제 확률은 p이다 / 대립가설 : 데이터의 실제 확률은 p가 아니다**

### ```binom.test(x, n, p, alternative, conf.level)``` 
- ```alternative``` : 

    (```"alternative"``` : 데이터의 실제확률은 p인지 검정 / ```"less"``` : 데이터의 실제확률이 p보다 작은 비율인가를 검정 / ```"greater"``` : : 데이터의 실제확률이 p보다 큰 비율인가를 검정)
- ```conf.level``` : 신뢰수준 

#### 작년 성공률이 0.2이고, 올해 성공률이 14 / 150이라면 
- p < 0.05 => 귀무가설 기각 => 작년 성공률과 올해 성공률에 차이가 존재

In [111]:
binom.test(14, 150, p = 0.2, alternative = "two.sided", conf.level = 0.95)


	Exact binomial test

data:  14 and 150
number of successes = 14, number of trials = 150, p-value = 0.0006735
alternative hypothesis: true probability of success is not equal to 0.2
95 percent confidence interval:
 0.05197017 0.15163853
sample estimates:
probability of success 
            0.09333333 


### 
### 단일집단 평균검정 (일표본 t검정)
### 절차
- **가설설정 => 유의수준 설정 => 검정통계량 및 유의확률 계산 => 귀무가설 기각여부 판단**
### 

### **1. 정규성 검정**
- **귀무가설 : 데이터는 정규분포를 따른다 / 대립가설 : 데이터는 정규분포를 따르지 않는다**
### ```shapiro.test(data)``` : 5000건 이하의 데이터 정규성 검정
### ```nortest::ad.test(data)``` : 5000건 이상의 데이터 정규성 검정

In [112]:
data <- na.omit(data)
shapiro.test(data$time)


	Shapiro-Wilk normality test

data:  data$time
W = 0.99137, p-value = 0.7242


- p > 0.05 => 귀무가설 기각 => 데이터는 정규분포를 따르지 않는다

### 
### 2. 평균검정 
- **귀무가설 : 데이터의 평균은 $\mu$이다 / 대립가설 : 데이터의 평균은 $\mu$이 아니다.**


### &nbsp;&nbsp; 2-1. 정규성 만족시 => ```t.test(x, alternative, mu, paired, var.equal, conf.level)``` 
### &nbsp;&nbsp; 2-2. 정규성 불만족시 => ```wilcox.test(x, alternative, mu, paired, conf.level)```

**```alternative = "greater```" => 귀무가설 : x의 평균이 $\mu$보다 크다**

**```alternative = "less```" => 귀무가설 : x의 평균이 $\mu$보다 작다**

In [113]:
t.test(data$time, mu = 6, alternative = "two.sided")


	One Sample t-test

data:  data$time
t = -4.8996, df = 108, p-value = 3.395e-06
alternative hypothesis: true mean is not equal to 6
95 percent confidence interval:
 5.377613 5.736148
sample estimates:
mean of x 
 5.556881 


In [114]:
wilcox.test(data$time, mu = 6, alternative = "two.sided")


	Wilcoxon signed rank test with continuity correction

data:  data$time
V = 1394, p-value = 5.509e-06
alternative hypothesis: true location is not equal to 6


### 
### 3. 검정통계량 및 유의확률 계산
### t검정통계량 : $t_0 = {{\bar{x}-\mu_0} \over {s / \sqrt{n}}} \sim t(df),\;\;\;\; df = n - 1$
- **표준오차 : ${s \over \sqrt{n}}$**

In [115]:
xbar <- mean(data$time) ; s = sd(data$time) ; n = data$time %>% length ; mu = 6
(xbar - mu) / (s / sqrt(n))
xbar + qt(0.975, df = n - 1) * s / sqrt(n)
xbar + qt(0.025, df = n - 1) * s / sqrt(n)

[1] -4.899606

[1] 5.736148

[1] 5.377613

# 
# 
## 이표본 검정

In [80]:
data <- read.csv("C:/Soongsil/R/빅데이터 R프로그래밍/Rwork-2nd/Part-III/two_sample.csv")
factor(data$method) %>% summary; factor(data$survey) %>% summary
table(data$method, data$survey)

1   2 
150 150

0   1 
 55 245

   
      0   1
  1  40 110
  2  15 135

### 
### 이표본 비율검정 : 두 집단 간 성공확률에 차이가 있는지를 검정
- **귀무가설 : 두 집단 간 성공확률은 동일하다 / 대립가설 : 두 집단 간 성공확률은 차이가 있다**

### ```prop.test(x, n, p, alternative, conf.level)```  

**```alternative = "greater```" => 귀무가설 : 첫 번째 비율이 두 번째 비율보다 크다**

**```alternative = "less```" => 귀무가설 : 첫 번째 비율이 두 번째 비율보다 작다**

In [81]:
prop.test(x = c(110, 135), n = c(150, 150), alternative = "two.sided", conf.level = 0.95)


	2-sample test for equality of proportions with continuity correction

data:  c(110, 135) out of c(150, 150)
X-squared = 12.824, df = 1, p-value = 0.0003422
alternative hypothesis: two.sided
95 percent confidence interval:
 -0.25884941 -0.07448392
sample estimates:
   prop 1    prop 2 
0.7333333 0.9000000 


- p < 0.05 => 귀무가설 기각 => data$method별 '1'의 비율 (성공확률)에는 차이가 있다

## 
### 이표본 평균검정 
### 대응표본 t검정 (전 / 후 비교)
- **귀무가설 : 두 집단의 모평균 간에는 차이가 없다 / 대립가설 : 두 집단의 모평균 간에는 차이가 있다**
- **가정 : 모집단의 관측값이 정규성을 만족**

### 절차
- **가설설정 => 유의수준 설정 => 검정통계량 및 유의확률 계산 => 귀무가설 기각여부 판단**
### 

### 1. 정규성 검정
### 2. 등분산 검정 (동질성 검정)
- **귀무가설 : 두 집단간 분포의 모양이 동질적이다 (등분산성 만족) / 대립가설 : 두 집단간 분포의 모양이 동질적이지 않다**
- **등분산의 의미 : 모집단에서 추출된 표본이 균등하게 추출된 의미**
- **이분산의 의미 : 추출된 표본이 특정 계층으로 편중되어 추출됨**


### &nbsp;&nbsp; 2-1. 정규성 만족시 => ```var.test(A, B)``` : F검정
### &nbsp;&nbsp;&nbsp;&nbsp; 검정통계량 $F = {{{\sum^{n_A}_{i=1} (X_{Ai} - \bar{X_A})^2} \over {{n_A}-1}} \over {{\sum^{n_B}_{i=1}(Y_{Bi} - \bar{Y_B})^2} \over {{n_B}-1}}} = {V_{X_A} \over V_{Y_B}} \sim F(df_A, df_B)$


### &nbsp;&nbsp; 2-1. 정규성 불만족시 => ```car::leveneTest(y, group)```
- ```y``` : 검정하는 변수
- ```group``` : 범주


In [180]:
data <- data.frame(before = c(7, 3, 4, 5, 2, 1, 6, 6, 5, 4),
                   after = c(8, 4, 5, 6, 2, 3, 6, 8, 6, 5))

In [184]:
shapiro.test(data$after) ; shapiro.test(data$before)


	Shapiro-Wilk normality test

data:  data$after
W = 0.94568, p-value = 0.6178



	Shapiro-Wilk normality test

data:  data$before
W = 0.96446, p-value = 0.8353


In [172]:
var.test(data$after, data$before)


	F test to compare two variances

data:  data$after and data$before
F = 1.0623, num df = 9, denom df = 9, p-value = 0.9297
alternative hypothesis: true ratio of variances is not equal to 1
95 percent confidence interval:
 0.2638616 4.2768349
sample estimates:
ratio of variances 
          1.062305 


- **F검정 검정통계량, 신뢰구간**

In [188]:
var_A <- var(data$after) ; var_B <- var(data$before)
n_A <- data$after %>% length ; n_B <- data$before %>% length

In [195]:
var_A / var_B
(var_A / var_B) / qf(0.975, df = n - 1, df2 = n - 1)
(var_A / var_B) * qf(0.975, df1 = n - 1, df2 = n - 1)

[1] 1.062305

[1] 0.2638616

[1] 4.276835

### 
### 3. 평균검정
### ```t.test(x, y, alternative, paired = TRUE, m)```
- ```paired``` : **대응표본 t검정 수행 여부 (TRUE : 대응표본 / FALSE : 독립표본)**
- ```m``` : 검정의 기본값. 대응표본 t검정에서는 모평균의 차이가 0인지를 검정하기에 => m = 0

**```alternative = "greater```" => 귀무가설 : 첫 번째 평균이 두 번째 평균보다 크다**

**```alternative = "less```" => 귀무가설 : 첫 번째 평균이 두 번째 평균보다 작다**

In [126]:
t.test(data$before, data$after, alternative = "two.sided", paired = TRUE)


	Paired t-test

data:  data$before and data$after
t = -4.7434, df = 9, p-value = 0.001054
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -1.4769046 -0.5230954
sample estimates:
mean of the differences 
                     -1 


# 
### 4. 검정통계량 및 유의확률 계산
### t검정통계량 : $t_0 = {{\bar{D} - (\mu_A - \mu_B)} \over {s_D / \sqrt{n}}} \sim t(df),\;\;\;\; df = n - 1$

- **$\bar{D}$** : 차이의 평균 / $s_D$ : 차이의 표준편차 / $\mu_A$ : 전 평균 / $mu_B$ : 후 평균 / $n$ = 표본크기
- **$\mu_A - \mu_B$의 표준오차 : $(\mu_A - \mu_B) \over {s_D / \sqrt{n}}$**

In [124]:
mu_AB <- mean(data$before - data$after) ; s_D <- sd(data$before - data$after) ; n = nrow(data) 

In [125]:
(mu_AB - 0) / (s_D / sqrt(n))
mu_AB + qt(0.975, df = n - 1) * dsd / sqrt(n)
mu_AB + qt(0.025, df = n - 1) * dsd / sqrt(n)

[1] -4.743416

[1] -0.5230954

[1] -1.476905

## 
### 이표본 평균검정 
### 독립표본 t검정 (AB test)
- **귀무가설 : 두 집단의 모평균 간에는 차이가 없다 / 대립가설 : 두 집단의 모평균 간에는 차이가 있다**
- **가정 : 모집단의 관측값이 정규성을 만족 / 두 개의 모집단은 서로 독립적 / 두 모집단의 등분산성**

### 절차
- **가설설정 => 유의수준 설정 => 검정통계량 및 유의확률 계산 => 귀무가설 기각여부 판단**
### 

### 1. 정규성 검정
### 2. 등분산 검정 (동질성 검정, F검정)

In [143]:
library(MASS)

In [159]:
shapiro.test(cats$Bwt)
shapiro.test(cats %>% filter(Sex == "M") %>% .$Bwt) ; shapiro.test(cats %>% filter(Sex == "F") %>% .$Bwt) 


	Shapiro-Wilk normality test

data:  cats$Bwt
W = 0.95188, p-value = 6.731e-05



	Shapiro-Wilk normality test

data:  cats %>% filter(Sex == "M") %>% .$Bwt
W = 0.97883, p-value = 0.119



	Shapiro-Wilk normality test

data:  cats %>% filter(Sex == "F") %>% .$Bwt
W = 0.89096, p-value = 0.0003754


- 전체 데이터는 정규성 불만족
- Sex가 "M"인 데이터는 정규성 만족, Sex가 "F"인 데이터는 정규성 불만족

In [161]:
var.test(cats %>% filter(Sex == "M") %>% .$Bwt, cats %>% filter(Sex == "F") %>% .$Bwt)
car::leveneTest(y = cats$Bwt, group = cats$Sex)


	F test to compare two variances

data:  cats %>% filter(Sex == "M") %>% .$Bwt and cats %>% filter(Sex == "F") %>% .$Bwt
F = 2.9112, num df = 96, denom df = 46, p-value = 0.0001157
alternative hypothesis: true ratio of variances is not equal to 1
95 percent confidence interval:
 1.723106 4.703057
sample estimates:
ratio of variances 
          2.911196 


,Df,F value,Pr(>F)
,<int>,<dbl>,<dbl>
group,1,19.43101,2.043529e-05
,142,NA,NA


### 
### 3. 평균검정
### ```t.test(x, y, alternative, paired = FALSE, m)```

In [162]:
t.test(cats %>% filter(Sex == "M") %>% .$Bwt, cats %>% filter(Sex == "F") %>% .$Bwt, 
       alternative = "two.sided", paired = FALSE)


	Welch Two Sample t-test

data:  cats %>% filter(Sex == "M") %>% .$Bwt and cats %>% filter(Sex == "F") %>% .$Bwt
t = 8.7095, df = 136.84, p-value = 8.831e-15
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.4177242 0.6631268
sample estimates:
mean of x mean of y 
 2.900000  2.359574 


# 
### 4. 검정통계량 및 유의확률 계산
### t검정통계량 : $t_0 = {{(\bar{X_A} - \bar{X_B}) - D_0} \over {s\;\sqrt{{1 \over n_A} + {1 \over n_B}}}} \sim t(df),\;\;\;\; df = n_1 + n_2 - 2$

- **$\bar{D}$** : 차이의 평균 / $\bar{X_A}$ : 표본A 평균 / $\bar{X_B}$ : 표본B 평균 / $n_A$ = A표본 크기 / $n_B$ B표본 크기
- **$\bar{X_A} - \bar{X_B}$의 표준오차 : ${s\;\sqrt{{1 \over n_A} + {1 \over n_B}}}$**

# 
# 
## 교차분석
### 분할표
### ```gmodels::CrossTable(x, y)```

In [202]:
data(survey, package = "MASS")

In [204]:
gmodels::CrossTable(survey$Sex, survey$Clap)


 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  235 

 
             | survey$Clap 
  survey$Sex |      Left |   Neither |     Right | Row Total | 
-------------|-----------|-----------|-----------|-----------|
      Female |        21 |        24 |        73 |       118 | 
             |     0.103 |     0.015 |     0.009 |           | 
             |     0.178 |     0.203 |     0.619 |     0.502 | 
             |     0.538 |     0.490 |     0.497 |           | 
             |     0.089 |     0.102 |     0.311 |           | 
-------------|-----------|-----------|-----------|-----------|
        Male |        18 |        25 |        74 |       117 | 
             |     0.103 |     0.015 |     0.009 |           | 
             |     0.154 |     0.214 |     0.632 |     0.498 | 
             

## 일원 카이제곱 검정
### 적합성 검정
### ```chisq.test(x, y, p)```

- **총합의 사전 설정 X**
- **관측값들의 분포 확인 (모집단 분포에 대한 가정이 맞는지 확인)**

####  귀무가설 : 실제 분포와 이론적 분포 간에는 차이가 없다 (두 분포는 일치) 
#### 대립가설 : 실제 분포와 이론적 분포 간에는 차이가 있다 (두 분포가 일치하지 않는다

### 검정통계량 $\chi^2 = \sum^k_{i=1} {{(O_i - E_i)^2} \over E_i} \sim \chi^2(k - 1)$
- **$O_i$ : 관찰도수** / **$E_i$ : 기대도수** / **k : 범주의 개수**
- **$\chi^2$통계량이 큰 경우 => 관찰도수와 기대도수의 차이가 크며, 적합도가 낮다 (두 분포가 일치하지 않음)**
- **$\chi^2$통계량이 작은 경우 => 관찰도수와 기대도수의 차이가 작으며, 적합도가 높다 (두 분포가 일치)**


In [200]:
(data <- table(survey$W.Hnd))


 Left Right 
   18   218 

In [201]:
chisq.test(data, p = c(0.2, 0.8))


	Chi-squared test for given probabilities

data:  data
X-squared = 22.581, df = 1, p-value = 2.015e-06


## 이원 카이제곱 검정 
### 독립성 검정 
### ```gmodels::CrossTable(x, y, chisq = TRUE)```

- **총합이 사전 설정**
- **모집단이 두 개의 변수에 의해 범주화되었을 때, 두 변수들 사이의 관계가 독립인지 아닌지를 검정**

####  귀무가설 : 두 변수 사이에는 연관이 없다 (독립)
#### 대립가설 : 두 변수 사이에는 연관이 있다 (종속)

### 검정통계량 $\chi^2 = \sum^r_{i = 1} \sum^c_{j=1} {{(O_{ij} - E_{ij})^2} \over E_{ij}} \sim \chi^2((R-1)(C-1)$

- **$E_{ij} = {{O_i \times O_j} \over n}$ : 기대빈도** / **$n$ : 전체 관측도수** / **$O_i$ : 행의 합** / **$O_j$ : 열의 합 / $O_{ij}$ : 관찰빈도**
- **R : 행의 수 / C : 열의 수**
- **$\chi^2$통계량이 큰 경우 => 두 변수 사이에는 연관이 있다 (종속)**
- **$\chi^2$통계량이 작은 경우 => 두 변수 사이에는 연관이 없다 (독립)**

In [209]:
gmodels::CrossTable(survey$W.Hnd, survey$Exer, chisq = TRUE)
# chisq.test(table(urvey$W.Hnd, survey$Exer))

Warning message in chisq.test(t, correct = FALSE, ...):
"Chi-squared approximation may be incorrect"



 
   Cell Contents
|-------------------------|
|                       N |
| Chi-square contribution |
|           N / Row Total |
|           N / Col Total |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  236 

 
             | survey$Exer 
survey$W.Hnd |      Freq |      None |      Some | Row Total | 
-------------|-----------|-----------|-----------|-----------|
        Left |         7 |         3 |         8 |        18 | 
             |     0.330 |     0.747 |     0.037 |           | 
             |     0.389 |     0.167 |     0.444 |     0.076 | 
             |     0.061 |     0.125 |     0.082 |           | 
             |     0.030 |     0.013 |     0.034 |           | 
-------------|-----------|-----------|-----------|-----------|
       Right |       107 |        21 |        90 |       218 | 
             |     0.027 |     0.062 |     0.003 |           | 
             |     0.491 |     0.096 |     0.413 |     0.924 | 
             

# 
### 동질성 검정
- **행합이 사전 설정**
- **모집단이 임의의 변수에 따라 R개의 속성으로 범주화 되었을 때, R개의 부분 모집단에서 추출한 각 표본인 C개의 범주화된 집단의 분포가 서로 동일한지를 검정**
- **계산법과 검증법은 독립성 검정과 동일**

####  귀무가설 : $P_{1j} = P_{2j} = ... = P_{rj}$ (모든 $P_{nj}$는 동일)
#### 대립가설 : $P_{nj}$중 다른 값이 하나 이상 존재 (Not $H_0$) 

### 검정통계량 $\chi^2 = \sum^r_{i = 1} \sum^c_{j=1} {{(O_{ij} - E_{ij})^2} \over E_{ij}} \sim \chi^2((R-1)(C-1)$

- **$E_{ij} = {{O_i \times O_j} \over n}$ : 기대빈도** / **$n$ : 전체 관측도수** / **$O_i$ : 행의 합** / **$O_j$ : 열의 합 / $O_{ij}$ : 관찰빈도**
- **R : 행의 수 / C : 열의 수**
- **$\chi^2$통계량이 큰 경우 => $P_{nj}$중 다른 값이 하나 이상 존재**
- **$\chi^2$통계량이 작은 경우 => 모든 $P_{nj}$는 동일**